In [1]:
from semantic_caching.core import SemanticCaching

In [ ]:
cache = SemanticCaching()

In [ ]:
question = 'How wold is the Nepal'
cache.ask(question, use_llm=True)

In [ ]:
question = 'How far is the Earth from Mars?'
cache.ask(question, use_llm=True)